# Final Project for CAP6307
## Cat Baker and Darshay Blount
### Fall 2020

In [1]:
# install modules
!pip install lyricsgenius
!pip install textblob
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /Users/catbaker3/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /Users/catbaker3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/catbaker3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/catbaker3/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/catbaker3/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/catbaker3/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [2]:
# import packages
from gensim.summarization import keywords
import spacy
import nltk
from nltk.corpus import stopwords
import pandas as pd
import lyricsgenius
import textblob
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# download spacy and import English
nltk.download('stopwords')
from spacy.lang.en import English

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/catbaker3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Import stopwords
stopwords = set(stopwords.words('english'))

In [4]:
# Set the nlp language to English
nlp = English()
nlp.max_length = 10000000

In [5]:
# Set up the token, id, and secret for the Lyric Genius API
client_id = 'jGXOq14YmSS490yTKQ7CPzu8l7ui5v4JnbBvhIhQV8VV74cSTpoFEqoKo5NqRuTT'
client_secret = 'MCDsDPsZLCO10I9_cAOVZp91OAIFgx-aUDmaZ79hHQ9jgnquVkDROF1NBjlIisu9X2yG86OrGz7nJF26VB5nQw'
access_token = 'vcwFJvy8po5FLkEkeTeSbhkbfidq43zZqTxoBNLnEbL7tltCTbDRiVeLJ68AL2rG'

In [6]:
# Set up the API information
genius = lyricsgenius.Genius(access_token)

In [7]:
# Import the Billboard Top 100 dataset
billboard_100 = pd.read_csv('billboardHot100_1999-2019.csv')

In [8]:
# Preview the dataframe
billboard_100.head(10)

,Unnamed: 0,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
0,1,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus
1,2,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,NaN,2019-07-06,"June 21, 2019",Pop,"Cashmere cat, Jack patterson, Charli xcx, Benn...",Senorita \nI love it when you call me senorita...,NaN
2,3,Billie Eilish,Bad Guy,3,2.0,13.0,2019-07-06,"March 29, 2019","Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...","Billie eilish, Finneas","bad guy \nWhite shirt now red, my bloody nose\...",NaN
3,4,Khalid,Talk,4,3.0,20.0,2019-07-06,"February 7, 2019","Synth-Pop,Pop","Howard lawrence, Guy lawrence, Khalid",Talk \nCan we just talk? Can we just talk?\nTa...,NaN
4,5,"Ed Sheeran, Justin Bieber",I Don't Care,5,2.0,7.0,2019-07-06,"May 10, 2019","Canada,UK,Dance,Dance-Pop,Pop","Ed sheeran, Justin bieber, Shellback, Max mart...",I Don't Care \nI'm at a party I don't wanna be...,NaN
5,6,Jonas Brothers,Sucker,6,1.0,17.0,2019-07-06,"March 1, 2019","Alternative Pop,Boy Band,Teen Pop,Pop-Rock,Pop","Ryan tedder, Louis bell, Frank dukes, Nick jon...",Sucker \nWe go together\nBetter than birds of ...,NaN
6,7,DaBaby,Suge,7,7.0,13.0,2019-07-06,"March 1, 2019","Trap,East Coast,Rap","Jetsonmade, Pooh beatz, Dababy","Suge \nPooh, you a fool for this one\nHa\nOh L...",NaN
7,8,Drake,Money In The Grave,8,7.0,2.0,2019-07-06,"June 15, 2019","Hip-Hop,Rap,Basketball,NBA,Canada","Asoteric, Ljay currie, Cydney christine, Rick ...","Money in the Grave \nYeah, okay\nLil CC on the...",Rick Ross
8,9,Chris Brown,No Guidance,9,9.0,3.0,2019-07-06,"June 8, 2019","Alternative R&;B,Hip-Hop,Rap,Pop,DMV,Canada,R&;B","Velous, J louis, Vinylz, Michee patrick lebrun...","No Guidance \nBefore I die I'm tryna fuck you,...",Drake
9,10,Post Malone,Wow.,10,2.0,27.0,2019-07-06,"December 24, 2018","Memes,Hip-Hop,Trap,Rap","Frank dukes, Billy walsh, Louis bell, Post malone","Wow \nSaid she tired of little money, need a b...",NaN


In [9]:
# Sort the dataframe by week ID (ascending)
billboard_100 = billboard_100.sort_values(by=['Week'])

In [10]:
billboard_100.head()

,Unnamed: 0,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
97224,97225,The Offspring,Why Don't You Get A Job?,100,74.0,15.0,1999-07-12,NaN,"Ska,Punk Rock,Rock",Dexter holland,Why Don't You Get a Job? \nMy friend's got a g...,NaN
97157,97158,Sugar Ray,Every Morning,27,3.0,28.0,1999-07-12,NaN,Rock,"David kahne, Sugar ray",Every Morning \nEvery morning there's a halo h...,NaN
97158,97159,Goo Goo Dolls,Slide,28,8.0,33.0,1999-07-12,NaN,"Alternative Rock,Rock",John rzeznik,Slide \nCould you whisper in my ear?\nThe thin...,NaN
97159,97160,Lonestar,Amazed,29,29.0,7.0,1999-07-12,NaN,"Pop,Country","Marv green, Chris lindsey, Aimee mayo",Amazed \nEvery time our eyes meet\nThis feelin...,NaN
97160,97161,Tim McGraw,Please Remember Me,30,10.0,16.0,1999-07-12,NaN,"Country,Rock","Will jennings, Rodney crowell",Please Remember Me \nWhen all our tears have r...,NaN


In [11]:
# Check for null values
billboard_100.isnull().sum()

# This makes since bc the null values belong to the song's first week on the Billboard chart

Unnamed: 0             0
Artists                0
Name                   0
Weekly.rank            0
Peak.position       7069
Weeks.on.chart      7069
Week                   0
Date               12247
Genre                  0
Writing.Credits     7213
Lyrics                 0
Features           75936
dtype: int64

In [16]:
# Filter to only show songs from the last ten years (November 2010-Dec 2019 (end of dataset))
billboard_100 = billboard_100.loc[billboard_100['Week'] > '2009-07-01']

In [17]:
billboard_100.head()

,Unnamed: 0,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
49402,49403,Dierks Bentley,Sideways,35,35.0,12.0,2009-07-02,"February 3, 2009",Country,"Dierks bentley, Jim beavers","Sideways \nHey girl, what's your name?\nIt's s...",NaN
49401,49402,Flo Rida,Sugar,34,5.0,14.0,2009-07-02,"March 31, 2009",Rap,"Steven battey, Carlos battey, Dj montay, Flo r...",Sugar \nDumm\nDouble dee double di\nDouble dee...,Wynter
49400,49401,Jason Mraz,I'm Yours,33,6.0,62.0,2009-07-02,"April 15, 2008","Adult Contemporary,Adult Alternative,Pop-Rock,...",Jason mraz,I'm Yours \nWell you done done me and you bet ...,NaN
49395,49396,Brad Paisley,Then,28,28.0,14.0,2009-07-02,"March 23, 2009",Country,"Ashley gorley, Brad paisley, Chris dubois",Then \nI remember trying not to stare\nThe nig...,NaN
49398,49399,Jordin Sparks,Battlefield,31,31.0,6.0,2009-07-02,"May 8, 2009","R&;B,Pop","Wayne wilkins, Sam watters, Louis biancaniello...","Battlefield \nDon't try to explain your mind, ...",NaN


In [27]:
billboard_100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49463 entries, 49402 to 0
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       49463 non-null  int64  
 1   Artists          49463 non-null  object 
 2   Name             49463 non-null  object 
 3   Weekly.rank      49463 non-null  int64  
 4   Peak.position    45402 non-null  float64
 5   Weeks.on.chart   45402 non-null  float64
 6   Week             49463 non-null  object 
 7   Date             47597 non-null  object 
 8   Genre            49463 non-null  object 
 9   Writing.Credits  47974 non-null  object 
 10  Lyrics           49463 non-null  object 
 11  Features         12949 non-null  object 
 12  Song_Artist      49463 non-null  object 
 13  Artist_Song      49463 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 5.7+ MB


In [29]:
# Create a unique identifier column 



<ipython-input-29-0418d80991d7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  billboard_100['Artist_Song'] = str(billboard_100['Artists'] + billboard_100['Name'])


In [ ]:
# Drop duplicate songs
billboard_100.drop_duplicates(subset='') #remove duplicate occurrences of songs

## Exploratory Data Analysis

In [ ]:
# Distribution of instances
max_instances = billboard_100.groupby('Song')['Instance'].max().reset_index()
max_instances['Instance'].value_counts()

In [ ]:
# Plot the distribution
plt.figure(figsize=(10,7))
sns.countplot(max_instances['Instance'])
plt.title('Distribution of the Number of Instances')
plt.xlabel('Number of Instances')
plt.ylabel('Number of Songs')

In [ ]:
# Distribution of weeks on the Billboard chart
max_weeks = billboard_100.groupby('Song')['Weeks on Chart'].max().reset_index()
max_weeks['Weeks on Chart'].value_counts()

In [ ]:
# Plot the distribution
plt.figure(figsize=(15,10))
sns.countplot(max_weeks['Weeks on Chart'])
plt.title('Distribution of the Number of Weeks on the Billboard 100 Chart')
plt.xlabel('Number of Weeks')
plt.ylabel('Number of Songs')

In [ ]:
# Look at the songs with the longest time on the charts
max_weeks.loc[max_weeks['Weeks on Chart'] >= 55].sort_values(by = 'Weeks on Chart', ascending=False)

In [ ]:
peak_position = billboard_100.groupby('Song')['Peak Position'].max().reset_index()
peak_position['Peak Position'].value_counts()

In [ ]:
# Import the Billboard Top 100 audio features Dataset
billboard_100_audio = pd.read_excel('billboard_100_audio.xlsx')

In [ ]:
# Remove duplicate songs
billboard_100_audio.drop_duplicates(subset='SongID', inplace=True)
billboard_100_audio.reset_index()

## Lyrics and Sentiment Analysis

In [ ]:
# Get the column names 
billboard_100.columns.tolist()

In [ ]:
# more column names
billboard_100_audio.columns.tolist()

In [ ]:
# Function to return the lyrics to each song
def get_lyrics(title, artist):
    try: 
        return genius.search_song(title, artist).lyrics
    except:
        return 'lyrics not found'

In [ ]:
# Function to return the sentiment score for each song
def lyrics_sentiment(lyrics):    
    analysis = TextBlob(lyrics)
    return analysis.sentiment.polarity

In [ ]:
# Text preprocessing
def preprocessing(text):
    document = nlp(text, disable=['ner','parser'])
    lemmas = [token.lemma_ for token in document]
    lemma_clean = [lemma for lemma in lemmas
                  if lemma.isalpha() and lemma not in stopwords]
    return ''.join(lemma_clean)

In [ ]:
# Keyword extraction
def keyword_extract(texts):
    xkeywords = []
    values = keywords(text=preprocessing(texts), split='\n', scores=True)
    for x in values[:10]:
        xkeywords.append(x[0])
    try:
        return xkeywords
    except:
        return 'there is no content'

In [ ]:
# Apply the function to get the lyrics
lyrics = billboard_100.apply(lambda row: get_lyrics(row['Song'], row['Performer']), axis=1)
billboard_100['Lyrics'] = lyrics
billboard_100 = billboard_100.drop(billboard_100[billboard_100['Lyrics'] == 'not found'].index) #drop rows where lyrics are not found on Genius

In [ ]:
#Use get_lyric_sentiment to get sentiment score for all the song lyrics
sentiment = billboard_100.apply(lambda row: lyrics_sentiment(row['Lyrics']), axis =1)
billboard_100['Sentiment'] = sentiment

In [ ]:
# Set the index to be the weekid
billboard_100.reset_index(inplace=True)

In [ ]:
#Set the index of the dataframe to the WeekID. This sets us up to resample dataframe based on time
billboard_100['WeekID'] = pd.to_datetime(billboard_100['WeekID'],infer_datetime_format=True)
billboard_100 = billboard_100.sort_values(by='WeekID')
billboard_100 = billboard_100.reset_index(drop=True)
billboard_100 = billboard_100.set_index('WeekID')

In [ ]:
#Resample dataframe lyrics by year. Get all the lyrics for every song for each year
#lyrics_resample = billboard_100['Lyrics'].resample('y').sum()

In [ ]:
#Resample dataframe sentiment column by year. This arranges the mean lyric sentiment for every year
billboard_100_resample = billboard_100['Sentiment'].resample('Y').mean()

In [ ]:
#Plot lyric sentiment over time
billboard_100_resample.plot(label='Sentiment', legend=True)
plt.title('Average Sentiment per Week over Time')
plt.xlabel('Week')
plt.ylabel('Sentiment')


# This shouldn't start at 2010
# the data starts at 2011 for this analysis

In [ ]:
#Use return_keywords function on lyrics_resample to get the top 20 keywords for each year
lyric_keywords = [keyword_extract(x[1]) for x in billboard_100_resample.iteritems()]